In [2]:
#Writing Replacement Function
import pandas as pd
import numpy as np
from utils.Dataframe_tools import PSD 
from utils import iterateSubDirFiles

#Setting the Values
IO='input files\KPBom.xlsx'

find_remove=[{
    '91840054':'99717392',
    '91840056':'99717485',
    '98441969':'92758523',
    '96698939':'92758528',     # in KPbom.xlsx 
    '96698968':'92758121',     # in KPbom.xlsx
    '96768950':'92758767',     # in KPbom.xlsx
    '99413027':'92758120',
    '1020-3/4':'1020-5/6',
    '1020-3_4':'1020-5_6',
    'Desc-1020-3_4-KP_4':'Desc-1020-5_6-KP_6',
    'Desc-1020-3_4-KP_3':'Desc-1020-5_6-KP_5'
},]

def find_and_replace(df:pd.DataFrame,to_replace,new_value)->None:
    df.replace(to_replace=to_replace,value=new_value,regex=True,inplace=True)

def change_psd_file(filename,function,*args,**kwargs)->list:
    sheet_list=pd.ExcelFile(filename).sheet_names
    df_dict={}
    for sheet in sheet_list:
        try:
            df=PSD(io=filename,sheet_name=sheet)
            function(df.psd_data,*args,**kwargs)
            df_dict.update({sheet:df})
        except TypeError:
            print(sheet.title()+" is not in PSD format.")
    return df_dict
        

dfs=change_psd_file(filename=IO,function=find_and_replace,to_replace=list(find_remove[0].keys()),new_value=list(find_remove[0].values()))



Info is not in PSD format.
Bom Breakdown is not in PSD format.
Misc is not in PSD format.
Seals is not in PSD format.
Couplings is not in PSD format.


In [3]:
case=dfs['Case']
data=case.psd_data
header=case.header_data

In [4]:


# for id in IDs:
#     data2.get_group(id)

endr=header[header.iloc[:,0].str.startswith("[")].index[0]

classes=header[:endr].values.tolist()
types=header.iloc[endr].tolist()
arr=[]
#Now I need to iterate over the classes
for row in classes:
    dct={}
    dct2={}
    new_data=data
    new_data.columns=row
    column_numbers = [x for x in range(new_data.shape[1])]
    for j,col in enumerate(row):
        if j==0:
            dct.update({'class':col})
        if str(col).strip().lower()=='nan':
            column_numbers .remove(j)
        else:
            if j==0:
                column_numbers.remove(0)
            else:
                dct2.update({col:types[j]})
        if str(col).strip().lower()=='id':
            data2=data.groupby(col)
            IDs=list(data2.groups.keys())
            dct.update({'instances':IDs})
    new_data=new_data.iloc[:, column_numbers]
    dct.update({'data':new_data})
    dct.update({'DataTypes':dct2})
    arr.append(dct)

    


In [20]:
import xml.etree.ElementTree as ET

"""
Instances
    |_Class
        |_Instance
            |_Class
                |_Ref
            |_Chartype
            |_attribute (data)

"""

"""Imports: PSD data with header removed """

num=0

#Outputs from my Code
psd_data=arr[num]['data'].iloc[:-1,:]
class_ref_name=arr[num]['class']
DataTypes=arr[num]['DataTypes']
Instances={x:ET.Element('instance',{'name':x}) for x in arr[num]['instances']} #List of elements that can appended to


# Iterate through dataframe in correct order to produce XML instance
root = ET.Element('class',{'name':class_ref_name})  # Root element


# Sets Class for each row/entry from PSD (Should be the same for all entries)
class_attr = {"name": class_ref_name}

# Sets chartype, which is always a doublebyte
chartype_attr = {"value": "doublebyte"}

for index, col in psd_data.iterrows():
    #Need to use the instance assoicated with the row being looked at
    instance=Instances[col['ID']]
    
    #We need to check to see if the instance already exists in root
    find_txt=f"instance[@name='{instance.attrib['name']}']"
    if root.find(find_txt)==None:
        root.append(instance)
        # Inserts Class for each instance in PSD
        class_entry = ET.SubElement(instance, "class")
        class_sub_entry = ET.SubElement(class_entry, "ref", attrib=class_attr)
        # Inserts chartype attribute
        chartype = ET.SubElement(instance, "chartype", chartype_attr)
    
    # # Loop through header dictionary for attribute types
    for k, v in  DataTypes.items():
        find_txt=f"attribute[@name='{k}']"
        # Need to insert if statement where pointers receive sub-child
        if v == "text":
            attr_dict = {"name": k, "type": v, "value": psd_data.at[index, k].__str__()}
            if instance.find(find_txt)==None:
                attr_entry = ET.SubElement(instance, "attribute", attr_dict) #This makes a new element each time and I want it to only make a new element
            else:
                attr_entry=instance.find(find_txt) 
        elif "pointer" in v: # Whats the difference between pointer and pointer-merge? May have to change this. 
            attr_dict = {"name": k, "type": v}
            if instance.find(find_txt)==None:
                attr_entry = ET.SubElement(instance, "attribute", attr_dict)
            else:
                attr_entry=instance.find(find_txt)
            pointer_dict = {"value": psd_data.at[index, k].__str__()}
            if instance.find(f"./attribute/ref[@value='{pointer_dict['value']}']")==None:
                pointer_sub_entry = ET.SubElement(attr_entry, "ref", attrib=pointer_dict)
  

In [6]:
instance.find('./attribute/ref').attrib['value']

'ConTab_BOM_KP_Case_0005'

In [7]:
a=instance.find("attribute[@name='PriceList']")
a[0]

<Element 'ref' at 0x0000019213291F80>

In [8]:
from xml.dom import minidom

def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = ET.tostring(elem, 'UTF-8')
    reparsed = minidom.parseString(rough_string)
    print(reparsed.toprettyxml(indent="  "))


In [21]:
prettify(root)

<?xml version="1.0" ?>
<class name="Price_BOM_KP_Case">
  <instance name="ConTab_BOM_KP_Case_0001">
    <class>
      <ref name="Price_BOM_KP_Case"/>
    </class>
    <chartype value="doublebyte"/>
    <attribute name="ID" type="pointer-merge">
      <ref value="ConTab_BOM_KP_Case_0001"/>
    </attribute>
    <attribute name="Model" type="text" value=":2095-1_2-KP:2095-1_2-KPV:2095-1_2-KPVS:"/>
    <attribute name="NozzleConfiguration" type="text" value="125# ANSI Flange"/>
    <attribute name="CaseMaterial" type="text" value="Cast Iron, ASTM-A48, CL 35"/>
    <attribute name="PacoMatlCode" type="text" value="C35"/>
    <attribute name="Coating" type="text" value="Coating_Standard"/>
    <attribute name="BOM" type="text" value="96698885"/>
    <attribute name="PriceID" type="pointer">
      <ref value="B100030"/>
    </attribute>
    <attribute name="LeadtimeID" type="pointer">
      <ref value="LT027"/>
    </attribute>
  </instance>
  <instance name="ConTab_BOM_KP_Case_0002">
    <cl

In [ ]:
#Converting to XML

import xml.etree.ElementTree as ET

root=ET.Element('FirepondSPCDatabase',attrib={'version':'3.0'}) #Creates the Root Node

with open('test.xml','wb') as doc:
    doc.write(ET.tostring(root,encoding='UTF-8',xml_declaration=True,short_empty_elements=False))



In [ ]:
new_data.iloc[:,[0,1]]